Los procedimientos almacenados son objetos de la base de datos y deben ser ejecutados explícitamente → llamados manualmente para que se ejecuten.
A diferencia de los triggers, que se activan automáticamente en respuesta a ciertos eventos.

Un procedimiento almacenado es un conjunto de instrucciones SQL que se almacenan en el servidor de la base de datos y pueden ser invocados en cualquier momento. 

Estos procedimientos pueden tomar parámetros y realizar diversas tareas, como recuperar datos, modificar registros, realizar cálculos complejos, etc.

Para ejecutar un procedimiento almacenado, generalmente se utiliza la sentencia "CALL" seguida del nombre del procedimiento y los argumentos necesarios, si los hay. Por ejemplo:

  * CALL nombre_del_procedimiento(argumento1, argumento2, ...);

También puedes ejecutar procedimientos almacenados desde tu aplicación utilizando el conector de MySQL correspondiente en el lenguaje de programación que estés utilizando.

ATENCIÓN: Los procedimientos almacenados ofrecen ventajas como la reutilización de código, la mejora del rendimiento y la seguridad, pero su uso también puede implicar una mayor complejidad y mantenimiento.

========================================================================================

In [41]:
import os
import pandas as pd
import mysql.connector
import configparser

In [42]:
# Leer las credenciales del archivo de configuración
config = configparser.ConfigParser()
config.read('config.ini')

user = config['database']['user']
password = config['database']['password']
host = config['database']['host']
database = config['database']['database']

# ==============================================================================

# Conectar a la base de datos
conexion = mysql.connector.connect(
    host=host,
    database=database,
    user=user,
    password=password
)

cursor = conexion.cursor()

# ==============================================================================

# ==============================================================================

# Asegurarse de aplicar los cambios en la base de datos
conexion.commit()

# Cerrar el cursor y la conexión
cursor.close()
conexion.close()

# Código MySQL

DELIMITER //

CREATE PROCEDURE InsertarFlujoReporte(
    IN mes_param INT,
    IN anio_param INT
)
BEGIN
    -- Eliminar registros existentes para el mes y año especificados
    DELETE FROM tbl_flujo_reporte WHERE mes = mes_param AND anio = anio_param;

    -- Insertar nuevos registros basados en los acumulados de NETO por ITEM_COD
    INSERT INTO tbl_flujo_reporte (id_item_cod, anio, mes, total_neto)
    SELECT 
        ITEM_COD,
        ANIO,
        MES,
        SUM(NETO) AS total_neto
    FROM tbl_transacciones
    WHERE MES = mes_param AND ANIO = anio_param
    GROUP BY ITEM_COD, ANIO, MES;

END //

DELIMITER ;

========================================================================================

DESCRIPCION

Este procedimiento almacenado: 
1. eliminará cualquier registro existente en la tabla "tbl_flujo_reporte" para el mes y año especificados y luego 
2. insertará nuevos registros basados en los acumulados de NETO por "ITEM_COD" para ese mes y año.

========================================================================================

* Para llamar a este procedimiento desde Jupyter Notebook con Python, puedes utilizar bibliotecas como mysql-connector-python o pymysql. 
* Ejemplo de cómo podrías llamar al procedimiento utilizando mysql-connector-python:

In [43]:
import mysql.connector
import configparser

In [44]:
# Leer las credenciales del archivo de configuración
config = configparser.ConfigParser()
config.read('config.ini')

user = config['database']['user']
password = config['database']['password']
host = config['database']['host']
database = config['database']['database']

# Abrir la base de datos: establecer la conexión
conn = mysql.connector.connect(user=user, password=password, host=host, database=database)

# crear un cursor
cursor = conn.cursor()

# seleccionar la base de datos
cursor.execute("USE flujo_caja_directo ")

# =====================================================================================================================

# Llamar al procedimiento almacenado
mes = 12  # Mes de ejemplo
anio = 2023  # Año de ejemplo
cursor.callproc("AcumulaFlujo", [mes, anio])

# =====================================================================================================================

# Confirmar los cambios
conn.commit()

# Cerrar el cursor y la conexión
cursor.close()
conn.close()

La tabla "tbl_flujo_reporte" debe resumir en cada "id_item" el contenido del campo NETO de la tabla "tbl_transacciones" según "ITEM_COD", "ANIO" y "MES"

Evaluar las siguientes opciones:
1. ir creando una columna en tabla "tbl_flujo_reporte" para resumir la información de cada mes/año
2. Modificar la estructura de la tabla "tbl_flujo_reporte" para que reciba un registro por cada acumulado de NETO por "ITEM_COD",   "ANIO" y "MES"

========================================================================================

1. **Crear una columna en la tabla "tbl_flujo_reporte" para resumir la información de cada mes/año**: 
* Esta opción implica agregar una nueva columna para cada mes/año en la tabla "tbl_flujo_reporte". Cada vez que se ingresen nuevos datos en la tabla "tbl_transacciones", tendrás que actualizar las columnas correspondientes en "tbl_flujo_reporte". 
* Esto podría ser eficiente si necesitas acceder rápidamente a los totales mensuales/año sin tener que realizar cálculos complejos en tiempo de consulta. 
* Sin embargo, podría volverse complicado si hay muchos meses/años diferentes o si la cantidad de datos es grande, ya que tendrías muchas columnas en la tabla "tbl_flujo_reporte".

2. **Modificar la estructura de la tabla "tbl_flujo_reporte" para que reciba un registro por cada acumulado de NETO por "ITEM_COD", "ANIO" y "MES"**: 
* En esta opción, en lugar de tener una columna para cada mes/año en "tbl_flujo_reporte", tendrías una fila por cada combinación de "ITEM_COD", "ANIO" y "MES", con una columna que almacena el total NETO correspondiente. 
* Esto sería más escalable y flexible, ya que no tendrías que modificar la estructura de la tabla cada vez que agregues un nuevo mes/año. 
* Además, permitiría consultas más flexibles y eficientes sobre los datos agregados.

En general, la opción 2 parece ser la más flexible y escalable, ya que te permite almacenar los totales NETO de manera estructurada y realizar consultas eficientes sobre ellos. Sin embargo, la elección final dependerá de tus necesidades específicas y de la complejidad de tu aplicación.

========================================================================================

Uno de los propósitos de la tabla "tbl_flujo_reporte" es que sirva como puente para tomar datos que alimentarán un reporte

* Si el propósito principal de la tabla "tbl_flujo_reporte" es alimentar un reporte, entonces la opción 2 parece ser la más adecuada. 
* Al tener una fila por cada combinación de "ITEM_COD", "ANIO" y "MES" con el total NETO correspondiente, puedes generar fácilmente los datos necesarios para el reporte utilizando consultas SQL simples.
* Además, al utilizar esta estructura, la tabla "tbl_flujo_reporte" actuará como un puente eficiente para obtener los datos agregados necesarios para el reporte, ya que los totales NETO estarán precalculados y disponibles en la tabla, lo que evitará la necesidad de realizar cálculos complejos en tiempo de consulta.

Por lo tanto, la opción 2 te proporcionará una estructura eficiente y escalable para alimentar el reporte, lo que facilitará el proceso de generación del mismo y garantizará un rendimiento óptimo, especialmente si estás trabajando con grandes volúmenes de datos.

Consulta para mostrar los procedimientos almacenados (stored procedures) en MySQL:

SHOW PROCEDURE STATUS;

========================================================================================

Consulta que muestra el código fuente completo del procedimiento almacenado (lógica y parámetros):

SHOW CREATE PROCEDURE nombre_del_procedimiento;

========================================================================================

Borrar:

DROP PROCEDURE IF EXISTS nombre_del_procedimiento;

========================================================================================